# **Challenge 5**: Foxhunting

Scenario:  You're walking through Las Vegas and you notice someone drops their Peplink SDX Enterprise Grade  someone carrying a router
In this challenge, you'll use a promiscuous WiFi dataset to determine the networks a samsung phone you saw at 6:36 was connected to, and try your best to locate where that person resides using a second dataset. staying at


## **Step 1**: Time Threshold
Using Pandas, set an approximate time treshold when you remembered spotting the device.  Keep in mind the WiGLE CSV format uses `UTC`.

In [ ]:
#### read wardriving csv file into Pandas dataframe
import pandas as pd
wd = pd.read_csv ('./Missoula-Warflight-08-21-2021.csv', delimiter = ',')

wd.drop(wd.index[wd["FirstSeen"] <= "2021-8-21 18:43:25"], inplace=True) # before takeoff
wd.drop(wd.index[wd["FirstSeen"] >= "2021-8-21 18:46:58"], inplace=True) # after landing

wd

## **Step 2**: Determining Possible Targets

Let's narrow down possible targets using our data on client devices and MAC addresses!  Here's a suggestion:

1. Print unique device manufacturers and narrow down vendors that would likely be inside a mobile device (Apple).
2. Plot individual client devices on a map (consider using FeatureGroup).
3. Identify your target based off their movement pattern!

*hint: we followed this person for close to a quarter mile.*

In [21]:
import folium

file = open('mac-vendors.txt', 'r', encoding="utf8")
mac_dict = {line[:8]: line[9:].strip() for line in file}
def getDeviceType(bssid): return mac_dict.get(bssid[:8], "Unknown")

bssid = "00:00:00:33:44:55"
print(getDeviceType(bssid))

wd['Manufacturer'] = wd['MAC'].apply(getDeviceType)
manufacturer_counts = wd['Manufacturer'].value_counts()

wd = wd[wd['Manufacturer'].str.contains('Apple')]

targets = folium.Map(wd[['Latitude', 'Longitude']].mean().values.tolist())
targets.fit_bounds([wd[['Latitude', 'Longitude']].min().values.tolist(), wd[['Latitude', 'Longitude']].max().values.tolist()])

for index, networks in wd.iterrows():
  folium.Marker((networks['Latitude'],networks['Longitude']), popup=networks['MAC'], icon=folium.Icon(color='darkblue', icon_color='white', icon='wifi', angle=0, prefix='fa')).add_to(targets)

targets


XEROX CORPORATION


# **Step 3**: Honing in
Thanks to probe requests, we can extract some hints where we might be able to trace down this person!  

Parse the second dataset containing promiscuous WiFi data, and use the MAC address from the previous step to determine past networks they've joined.


In [31]:
pdata = pd.read_csv ('./Missoula-Warflight-08-21-2021.csv', delimiter = ',')

pdata = pdata[pdata['MAC'] == '90:72:40:10:1B:CA']['SSID']


probes = pdata.unique()
probes

array(["Dennis's Wi-Fi Network"], dtype=object)

## **Step 4**: Open Source Intelligence
Now that you can see what networks they've joined in the past, we can use context clues to determine places they visit often!  

Try to determine as much information as you can!  You don't need to write code for this example.  Bonus point if you can find their hometown / employer.

*Hint: Use WiGLE.net!*

In [ ]:
## This example doesn't need code, but can you figure out potential places this person has visited?

## **Step 5**: Taking it Further
You decide to take things a step further and fly all the way out to their hometown and wardrive our friend's corporate office to ensure that everyone is using the best security standards before returning their computer.  

What wireless assets are on their network?  Can you find the names of some of our friend's coworkers?